PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [3]:
!pip install pyspark==3.3.1 py4j==0.10.9.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=3b03dd22b40da7b29690a2e6b91f2f6cd401f30e4f5e0cf0a5591bd2813681af
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession /
    .builder /
    .appName("Boston Housing Linear Regression example") /
    .getOrCreate()

# 보스턴 주택 가격 예측 모델 만들기




In [5]:
spark

In [6]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv

--2023-01-30 11:28:37--  https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.218.228.137, 52.92.176.202, 3.5.81.11, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.218.228.137|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36240 (35K) [text/csv]
Saving to: ‘boston_housing.csv’

boston_housing.csv  100%[===================>]  35.39K  --.-KB/s    in 0.09s   

2023-01-30 11:28:38 (416 KB/s) - ‘boston_housing.csv’ saved [36240/36240]



In [7]:
!ls -tl

total 40
drwxr-xr-x 1 root root  4096 Jan 26 14:33 sample_data
-rw-r--r-- 1 root root 36240 Jan 31  2021 boston_housing.csv


In [2]:
data = spark.read.csv('./data/boston_housing.csv', header=True, inferSchema=True)

In [3]:
data.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [4]:
data.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

## 피쳐 벡터를 만들기

In [5]:
# VectorAssembler를 import해서
# input이 되는 컬럼 이름은 feature_columns이고
# outputCol을 만들어라, 그것의 이름은 features로 줘라

from pyspark.ml.feature import VectorAssembler

feature_columns = data.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

In [6]:
feature_columns

['crim',
 'zn',
 'indus',
 'chas',
 'nox',
 'rm',
 'age',
 'dis',
 'rad',
 'tax',
 'ptratio',
 'b',
 'lstat']

In [9]:
# assembler로 dataframe을 주면, 출력으로 또다른 데이터프레임이 만들어지고
# 새로운 컬럼이 하나 붙는다
# 지정된 인풋컬럼들이 하나의 벡터로 묶여서 이 이름의 피처로 들어간다.
# transform이라는 함수는 하나의 데이터프레임을 입력을 받아서, 거기에 새로운 컬럼을 추가해주는 기능을 한다
# 앞의 transform의 앞에 붙은 변수의 클래스에 따라서 변형이 달라진다
data_2 = assembler.transform(data)

In [11]:
data_2.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|            features|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|[0.02985,0.0,2.18...|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5

## 훈련용과 테스트용 데이터를 나누고 Linear Regression 모델을 하나 만든다

In [12]:
train, test = data_2.randomSplit([0.7, 0.3])

In [13]:
from pyspark.ml.regression import LinearRegression

# featureCol에 이미 하나를 묶어놓은게 있기 때문에 features라는 벡터가 들어간 걸 input 컬럼으로 주고
# medv가 예측해야하는 레이블 값
algo = LinearRegression(featuresCol="features", labelCol="medv")
model = algo.fit(train)

## 모델 성능 측정

In [14]:
evaluation_summary = model.evaluate(test)

In [15]:
evaluation_summary

In [16]:
evaluation_summary.meanAbsoluteError

3.2956781534060786

In [17]:
evaluation_summary.rootMeanSquaredError

4.88953586941829

In [18]:
evaluation_summary.r2

0.722761515866863

## 모델 예측값 살펴보기 

In [19]:
# test 데이터를 model에 넘기면 predictions라는 컬럼을 넘겨준다
predictions = model.transform(test)

In [21]:
predictions.show()

+-------+----+-----+----+------+-----+----+-------+---+---+-------+------+-----+----+--------------------+------------------+
|   crim|  zn|indus|chas|   nox|   rm| age|    dis|rad|tax|ptratio|     b|lstat|medv|            features|        prediction|
+-------+----+-----+----+------+-----+----+-------+---+---+-------+------+-----+----+--------------------+------------------+
|0.01301|35.0| 1.52|   0| 0.442|7.241|49.3| 7.0379|  1|284|   15.5|394.74| 5.49|32.7|[0.01301,35.0,1.5...|30.651722574881468|
|0.01439|60.0| 2.93|   0| 0.401|6.604|18.8| 6.2196|  1|265|   15.6| 376.7| 4.38|29.1|[0.01439,60.0,2.9...| 31.20542003271651|
|0.01538|90.0| 3.75|   0| 0.394|7.454|34.2| 6.3361|  3|244|   15.9|386.34| 3.11|44.0|[0.01538,90.0,3.7...|  36.9170970302615|
|0.01709|90.0| 2.02|   0|  0.41|6.728|36.1|12.1265|  5|187|   17.0|384.46|  4.5|30.1|[0.01709,90.0,2.0...|25.149711292237377|
| 0.0187|85.0| 4.15|   0| 0.429|6.516|27.7| 8.5353|  4|351|   17.9|392.43| 6.36|23.1|[0.0187,85.0,4.15...| 24.73515781

In [23]:
predictions.select(predictions.columns[13:]).show()

+----+--------------------+------------------+
|medv|            features|        prediction|
+----+--------------------+------------------+
|32.7|[0.01301,35.0,1.5...|30.651722574881468|
|29.1|[0.01439,60.0,2.9...| 31.20542003271651|
|44.0|[0.01538,90.0,3.7...|  36.9170970302615|
|30.1|[0.01709,90.0,2.0...|25.149711292237377|
|23.1|[0.0187,85.0,4.15...| 24.73515781443145|
|20.1|[0.01965,80.0,1.7...|19.768589104289582|
|50.0|[0.02009,95.0,2.6...|42.706355855947585|
|42.3|[0.02177,82.5,2.0...|  36.8990181993991|
|16.5|[0.02498,0.0,1.89...| 22.68601534894256|
|28.7|[0.02985,0.0,2.18...|25.286775304203854|
|20.6|[0.03306,0.0,5.19...|21.961880964684745|
|19.5|[0.03427,0.0,5.19...| 19.97432716073832|
|48.5|[0.0351,95.0,2.68...|41.528559818803636|
|20.9|[0.03548,80.0,3.6...|21.353369437453743|
|27.9|[0.03615,80.0,4.9...|30.725191932813246|
|24.8|[0.03659,25.0,4.8...|25.791686718378998|
|34.6|[0.03768,80.0,1.5...| 34.89405523339659|
|22.0|[0.03932,0.0,3.41...| 27.27010283525874|
|28.0|[0.0411

In [24]:
model.save("./data/boston_housing_model")

In [26]:
!ls boston_housing_model

data  metadata


In [27]:
!ls -tl boston_housing_model

total 8
drwxr-xr-x 2 root root 4096 Jan 30 11:28 data
drwxr-xr-x 2 root root 4096 Jan 30 11:28 metadata


In [28]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [30]:
model_save_name = "boston_housing_model"
path = F"/content/gdrive/My Drive/boston_housing_model" 
model.save(path)

In [28]:
from pyspark.ml.regression import LinearRegressionModel
path = 'C:/beginner-spark-programming-with-pyspark/chapter4/data/boston_housing_model'
loaded_model = LinearRegressionModel.load(path)  # "boston_housing_model")

Py4JJavaError: An error occurred while calling o219.load.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/C:eginner-spark-programming-with-pyspark/chapter4/dataoston_housing_model/data/part-00000-20854e40-079a-4c80-9b53-494164c654c6-c000.snappy.parquet/metadata
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:287)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:291)
	at scala.Option.getOrElse(Option.scala:189)
	

In [27]:
predictions2 = loaded_model.transform(test)

NameError: name 'loaded_model' is not defined

In [33]:
predictions2.select(predictions.columns[13:]).show()

+----+--------------------+------------------+
|medv|            features|        prediction|
+----+--------------------+------------------+
|29.1|[0.01439,60.0,2.9...|31.863971610226315|
|24.5|[0.01501,80.0,2.0...|27.800912626874748|
|44.0|[0.01538,90.0,3.7...|38.081559279682246|
|23.9|[0.02543,55.0,3.7...|27.969747095640912|
|30.8|[0.02763,75.0,2.9...|31.575873666286917|
|25.0|[0.02875,28.0,15....|29.689655131070666|
|28.5|[0.03502,80.0,4.9...|33.855798290686124|
|45.4|[0.03578,20.0,3.3...|39.495929078055646|
|22.0|[0.03932,0.0,3.41...|27.678841182848963|
|33.3|[0.04011,80.0,1.5...|36.633177721713174|
|22.9|[0.04203,28.0,15....| 29.65081329317448|
|18.2|[0.04301,80.0,1.9...|14.312466963142263|
|20.5|[0.04337,21.0,5.6...| 24.52586127306892|
|19.8|[0.04544,0.0,3.24...|21.560592700808435|
|23.3|[0.0456,0.0,13.89...|27.974279873488676|
|17.1|[0.05023,35.0,6.0...|20.027398326196785|
|22.5|[0.05188,0.0,4.49...| 22.19486700208488|
|36.1|[0.05515,33.0,2.1...| 33.36611368642783|
|29.0|[0.0556